In [1]:
import pandas as pd
import math
import datetime
import statistics
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from pyFTS.benchmarks import Measures

In [3]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
data = data.loc[0: : 10]
data.dropna()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
10,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
20,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
30,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
40,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503860,1.621417,0.004183,0.000033,0.441950,0.645467,0.041767,0.129317,0.008067,0.012883,0.000317,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503870,0.896250,0.004150,0.000233,0.496633,0.063000,0.042317,0.123800,0.007850,0.012433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503880,1.311617,0.004167,0.000017,0.359167,0.645433,0.041917,0.005133,0.008167,0.013083,0.000650,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503890,2.059367,0.003383,0.000033,0.153367,0.643300,0.042050,0.005117,0.008283,0.013267,0.000433,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [6]:
# Create a lag feature
y_appliance = data['use [kW]']
var = pd.DataFrame(y_appliance)
dataframe = pd.concat([var.shift(1), var], axis=1)
dataframe.columns = ['t', 't+1']

In [7]:
dataframe

,t,t+1
0,NaN,0.932833
10,0.932833,1.585083
20,1.585083,0.533817
30,0.533817,1.190950
40,1.190950,1.582083
...,...,...
503870,1.621417,0.896250
503880,0.896250,1.311617
503890,1.311617,2.059367
503900,2.059367,1.522583


In [8]:
# Create a baseline model (Naive model)
def model_persistence(x):
  return x

def test_persistence(test_X):
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
    return predictions

In [9]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [11]:
result = {
     "window": [],
     "rmse": [],
     "nrmse": [],
     "mape": [],
     "mae": [],
     "smape": []
}

tam = len(dataframe)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(dataframe[1:tam], windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        print('-' * 20)
        print(f'training window {(ct)}')
        
        train_X, train_y = ttrain.iloc[:,0], ttrain.iloc[:,1]
        test_X, test_y = ttest.iloc[:,0], ttest.iloc[:,1]
        
        forecast = test_persistence(test_X)
        
        #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(test_y,forecast)
        nrmse = cal_nrmse(rmse, test_y)
        mape = Measures.mape(test_y,forecast)
        smape = Measures.smape(test_y,forecast)
        mae = mean_absolute_error(test_y, forecast)
        r2 = r2_score(test_y, forecast)
        
        result["rmse"].append(rmse)
        result["nrmse"].append(nrmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["mae"].append(mae)
        result["window"].append(ct)
        
measures = pd.DataFrame(result)

--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432
--------------------
training window 15111
--------------------
training window 16790
--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259
--------------------
training window 36938
--------------------

In [12]:
measures

,window,rmse,nrmse,mape,mae,smape
0,0,0.739989,0.176173,52.328839,0.506135,21.950706
1,1679,0.702151,0.170544,86.802856,0.522223,26.000354
2,3358,0.490642,0.174691,71.544698,0.326182,22.173355
3,5037,0.632647,0.126388,51.579280,0.472395,19.037665
4,6716,0.529567,0.145708,103.781574,0.382618,24.744452
5,8395,0.370077,0.138901,42.883802,0.188634,16.595212
6,10074,0.599393,0.138593,80.843168,0.413974,24.546264
7,11753,0.485848,0.141865,31.911668,0.227886,13.135678
8,13432,0.573489,0.131873,70.957319,0.298557,19.978480
9,15111,0.464564,0.081620,40.792070,0.212299,15.640636


In [13]:

final_result = {
     "rmse": [],
     "nrmse": [],
     "mape": [],
     "mae": [],
     "smape": [],
}

final_result["rmse"].append(statistics.mean(measures['rmse']))
final_result["nrmse"].append(statistics.mean(measures['nrmse']))
final_result["mape"].append(statistics.mean(measures['mape']))
final_result["smape"].append(statistics.mean(measures['smape']))
final_result["mae"].append(statistics.mean(measures['mae']))
        
final_measures_persistence = pd.DataFrame(final_result)

print("Mean Statistics Persistence: ")
final_measures_persistence

Mean Statistics Persistence: 


,rmse,nrmse,mape,mae,smape
0,0.846188,0.152791,251.683677,0.468115,21.237035
